In [1]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

In [17]:
epochs = 10
range_min = -99999
range_max = 99999
batch_size = 4 
num_classes = 2
num_train_samples = 100
num_test_samples = 100
num_digits = 5

In [18]:
def sep(x,num_digits):
    x = np.array([int(i) for i in str(x)])
    while (x.shape[0] < num_digits):
        x = np.insert(x,0,0)
    return x

In [19]:
y_train = np.random.randint(low = 0,high = range_max+1,size = num_train_samples)
x_train = np.array([sep(x,num_digits) for x in y_train])

In [20]:
train_data = torch.from_numpy(x_train).type(torch.FloatTensor)
train_labels = torch.from_numpy(y_train).type(torch.LongTensor)
# test_data = torch.from_numpy(x_test).type(torch.FloatTensor)
# test_labels = torch.from_numpy(y_test).type(torch.LongTensor)

In [6]:
# This model will join a list of digits to a number
# input: 1 input - (num_digits,)
# output: 1   

class Net_merge(nn.Module):
    def __init__(self):
        super(Net_merge,self).__init__()
        self.layers = []
        
    def forward(self,x):
        n = x.shape[-1]
        i = 0 
        add_list = []
        ones = []
        while(n>=1):
            temp = x[:,n-1:n]
            self.layers.append(nn.Linear(1,1,bias=False))
            self.layers[-1].weight.data = torch.tensor([[10**i]]).type(torch.FloatTensor)
            out = self.layers[-1](temp)
            i +=1
            n-=1
            add_list.append(out)
            ones.append(1)
        
        self.layers.append(nn.Linear(x.shape[-1],1,bias=False))
        self.layers[-1].weight.data = torch.tensor([ones]).type(torch.FloatTensor)
        concat = torch.cat(add_list,-1)
        out = self.layers[-1](concat)
        return out
    
    def predict(self,x):
        return self.forward(x).type(torch.FloatTensor)

In [7]:
net = Net_merge()

In [8]:
# print(train_data[0:10])
print(train_labels[0:10])
net.forward(train_data[0:10])

tensor([ 21, 604, 302,  21, 251, 231, 215, 181, 257, 325])


tensor([[ 21.],
        [604.],
        [302.],
        [ 21.],
        [251.],
        [231.],
        [215.],
        [181.],
        [257.],
        [325.]], grad_fn=<MmBackward>)

In [21]:
out = net.forward(train_data)
temp = train_labels.numpy()
out = out.detach().numpy()
correct = 0
for i in range(out.shape[0]):
    if out[i] == temp[i]:
        correct+=1
    else:
        print('wrong example :',i)
        print(train_data[i])
        print(train_labels[i])
        print(out[i])
print(correct)

100


In [16]:
torch.save(net,'model_merge')

C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net_merge. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
